In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
# Load data for Tram
data = pd.read_csv("/Users/tbhuynh/Desktop/UNC_PHD/BIOS635/Final Project/train.csv")
submission = pd.read_csv("/Users/tbhuynh/Desktop/UNC_PHD/BIOS635/Final Project/test.csv")

# Save the 'id' column from submission before dropping
submission_ids = submission['id'].copy()

# Drop 'id' and 'Name' from both datasets
data.drop(columns=['id', 'Name'], inplace=True)
submission.drop(columns=['id', 'Name'], inplace=True)


In [16]:
# Load data for Ashley
#data = pd.read_csv("C:/Users/ashle/OneDrive/Documents/Courses/UNC Courses/BIOS 635 - ML/Final/train.csv")
#submission = pd.read_csv("C:/Users/ashle/OneDrive/Documents/Courses/UNC Courses/BIOS 635 - ML/Final/test.csv")

# Data Preprocessing

In [11]:
# Examine data
print(data.shape)
print(submission.shape)

data.head()

(140700, 18)
(93800, 17)


,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [12]:
# Examining missing data for training set

# Calculate percentage of NaNs per column
nan = data.isna().mean() * 100

for col, i in nan.items():
    print(f"{col}: {i:.2f}% missing")

Gender: 0.00% missing
Age: 0.00% missing
City: 0.00% missing
Working Professional or Student: 0.00% missing
Profession: 26.03% missing
Academic Pressure: 80.17% missing
Work Pressure: 19.84% missing
CGPA: 80.17% missing
Study Satisfaction: 80.17% missing
Job Satisfaction: 19.84% missing
Sleep Duration: 0.00% missing
Dietary Habits: 0.00% missing
Degree: 0.00% missing
Have you ever had suicidal thoughts ?: 0.00% missing
Work/Study Hours: 0.00% missing
Financial Stress: 0.00% missing
Family History of Mental Illness: 0.00% missing
Depression: 0.00% missing


Academic Pressure, CGPA, and Study Satisfaction all have substantial amount of missing data (80.17%). Replacing it with the mean may introduce bias. 

In [13]:
# Fill numerical variables with median
print(f"Number of rows with NaN in Train: {data.isnull().any(axis=1).sum()}")

for col in data.select_dtypes(include=['number']).columns:  
    data[col].fillna(data[col].median(), inplace=True)

# Fill categorical variables with mode
for col in data.select_dtypes(include=['object']).columns:  
    data[col].fillna(data[col].mode()[0], inplace=True)
    
print(f"Number of rows with NaN in Train now: {data.isnull().any(axis=1).sum()}")    

Number of rows with NaN in Train: 140700
Number of rows with NaN in Train now: 0


In [15]:
# Fill submission numerical with median from training data
for col in submission.select_dtypes(include=['number']).columns:  
    submission[col].fillna(data[col].median(), inplace=True)

# Fill submission categorical with mode from training data
for col in submission.select_dtypes(include=['object']).columns:  
    submission[col].fillna(data[col].mode()[0], inplace=True)

# Doing one-hot encoding
# Convert categorical variables to dummy/indicator variables
data = pd.get_dummies(data, drop_first=True)
submission = pd.get_dummies(submission, drop_first=True)

# Ensure submission columns align with training features
submission = submission.reindex(columns=data.columns.drop('Depression'), fill_value=0)

assert set(submission.columns) == set(data.columns) - {'Depression'}, "Mismatch in columns!"
print(f"Number of rows with NaN in Test now: {submission.isnull().any(axis=1).sum()}")    

Number of rows with NaN in Test now: 0


# Models